# 2. Mã Hóa Dữ Liệu Train/Test Đã Chia

Notebook này mã hóa:
- **Train set đã augment** từ `train_augmented.csv`
- **Test set gốc (không augment)** từ `test_original.csv`

**Quan trọng:** Vocabulary sẽ được xây dựng từ TRAIN set, sau đó áp dụng cho cả train và test.

## 2.1. Import Thư Viện

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import os
from collections import Counter

print('Đã import thành công các thư viện!')

✅ Đã import thành công các thư viện!


## 2.2. Cấu Hình

In [ ]:
# Đường dẫn input
DATA_DIR = './split_augmented_data'
TRAIN_PATH = os.path.join(DATA_DIR, 'train_augmented.csv')
TEST_PATH = os.path.join(DATA_DIR, 'test_original.csv')

# Đường dẫn output
OUTPUT_DIR = './encoded_split_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Tên cột
TEXT_COLUMN = 'review'
LABEL_COLUMN = 'sentiment'

# Tham số encoding
MIN_WORD_FREQ = 2  # Từ phải xuất hiện ít nhất 2 lần trong TRAIN
MAX_VOCAB_SIZE = 50000  # Giới hạn kích thước vocabulary

print(f'Input:')
print(f'   Train (augmented): {TRAIN_PATH}')
print(f'   Test (original): {TEST_PATH}')
print(f'Output: {OUTPUT_DIR}/')
print(f'Min word frequency: {MIN_WORD_FREQ}')
print(f'Max vocab size: {MAX_VOCAB_SIZE}')

📁 Input:
   Train (augmented): ./split_augmented_data\train_augmented.csv
   Test (original): ./split_augmented_data\test_original.csv
📁 Output: ./encoded_split_data/
⚙️  Min word frequency: 2
⚙️  Max vocab size: 50000


## 2.3. Load Dữ Liệu

In [ ]:
print('Đang load dữ liệu...\n')

# Load train (đã augment)
train_df = pd.read_csv(TRAIN_PATH)
print(f'Train set: {len(train_df):,} samples (đã augment)')
print(f'   Phân phối nhãn:')
print(train_df[LABEL_COLUMN].value_counts().sort_index())

# Load test (nguyên bản)
test_df = pd.read_csv(TEST_PATH)
print(f'\nTest set: {len(test_df):,} samples (KHÔNG augment)')
print(f'   Phân phối nhãn:')
print(test_df[LABEL_COLUMN].value_counts().sort_index())

📂 Đang load dữ liệu...

✅ Train set: 59,995 samples (đã augment)
   Phân phối nhãn:
sentiment
negative    29997
positive    29998
Name: count, dtype: int64

✅ Test set: 10,000 samples (KHÔNG augment)
   Phân phối nhãn:
sentiment
negative    5000
positive    5000
Name: count, dtype: int64


## 2.4. Tiền Xử Lý Text

In [ ]:
def preprocess_text(text):
    """Tiền xử lý văn bản"""
    if pd.isna(text):
        return []
    
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'[^a-z0-9\s]', ' ', text)  # Keep letters and numbers
    text = re.sub(r'\b\d+\b', '', text)  # Remove standalone numbers
    text = ' '.join(text.split())  # Remove extra whitespace
    
    return text.split()

print('Đã định nghĩa hàm preprocess_text')

✅ Đã định nghĩa hàm preprocess_text


In [ ]:
# Tiền xử lý train và test
print('Đang tiền xử lý texts...\n')

train_df['tokens'] = train_df[TEXT_COLUMN].apply(preprocess_text)
test_df['tokens'] = test_df[TEXT_COLUMN].apply(preprocess_text)

# Thống kê train
train_lengths = train_df['tokens'].apply(len)
print(f'Train - Thống kê độ dài:')
print(f'   Min: {train_lengths.min()}, Max: {train_lengths.max()}')
print(f'   Mean: {train_lengths.mean():.1f}, Median: {train_lengths.median():.0f}')
print(f'   P95: {train_lengths.quantile(0.95):.0f}')

# Thống kê test
test_lengths = test_df['tokens'].apply(len)
print(f'\nTest - Thống kê độ dài:')
print(f'   Min: {test_lengths.min()}, Max: {test_lengths.max()}')
print(f'   Mean: {test_lengths.mean():.1f}, Median: {test_lengths.median():.0f}')
print(f'   P95: {test_lengths.quantile(0.95):.0f}')

# Loại bỏ samples rỗng
train_df = train_df[train_df['tokens'].apply(len) > 0].reset_index(drop=True)
test_df = test_df[test_df['tokens'].apply(len) > 0].reset_index(drop=True)

print(f'\nSau khi loại bỏ samples rỗng:')
print(f'   Train: {len(train_df):,} samples')
print(f'   Test: {len(test_df):,} samples')

🔄 Đang tiền xử lý texts...

📏 Train - Thống kê độ dài:
   Min: 1, Max: 2485
   Mean: 183.9, Median: 128
   P95: 516

📏 Test - Thống kê độ dài:
   Min: 6, Max: 2151
   Mean: 233.2, Median: 175
   P95: 597

✅ Sau khi loại bỏ samples rỗng:
   Train: 59,995 samples
   Test: 10,000 samples


## 2.5. Xây Dựng Vocabulary từ TRAIN

In [ ]:
# QUAN TRỌNG: Chỉ xây dựng vocabulary từ TRAIN set
print('Đang xây dựng vocabulary từ TRAIN set...\n')

word_counts = Counter()
for tokens in train_df['tokens']:
    word_counts.update(tokens)

print(f'Tổng số từ unique trong TRAIN: {len(word_counts):,}')
print(f'Tổng số từ (bao gồm lặp): {sum(word_counts.values()):,}')

# Lọc từ theo tần suất
filtered_words = {word: count for word, count in word_counts.items() 
                 if count >= MIN_WORD_FREQ}

print(f'\nSau khi filter (freq >= {MIN_WORD_FREQ}): {len(filtered_words):,}')

# Giới hạn vocab size
if len(filtered_words) > MAX_VOCAB_SIZE:
    most_common = sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:MAX_VOCAB_SIZE]
    vocab_words = [word for word, _ in most_common]
else:
    vocab_words = list(filtered_words.keys())

print(f'Vocab size cuối cùng: {len(vocab_words):,}')
# Tạo word2idx mapping
word2idx = {'<PAD>': 0, '<UNK>': 1}
for word in vocab_words:
    word2idx[word] = len(word2idx)

idx2word = {idx: word for word, idx in word2idx.items()}

print(f'\nVocabulary:')
print(f'   Size: {len(word2idx):,}')
print(f'   Special tokens: <PAD> (0), <UNK> (1)')

# Top words
print(f'\nTop 15 từ phổ biến nhất trong TRAIN:')
for i, (word, count) in enumerate(sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:15], 1):
    print(f'   {i:2d}. {word:15s} ({count:,} lần)')

🔄 Đang xây dựng vocabulary từ TRAIN set...

📊 Tổng số từ unique trong TRAIN: 103,256
📊 Tổng số từ (bao gồm lặp): 11,033,048

📊 Sau khi filter (freq >= 2): 62,442
📊 Vocab size cuối cùng: 50,000

✅ Vocabulary:
   Size: 50,002
   Special tokens: <PAD> (0), <UNK> (1)

📝 Top 15 từ phổ biến nhất trong TRAIN:
    1. the             (628,968 lần)
    2. a               (308,037 lần)
    3. and             (305,512 lần)
    4. of              (271,542 lần)
    5. to              (249,209 lần)
    6. is              (196,985 lần)
    7. it              (184,850 lần)
    8. i               (177,146 lần)
    9. in              (174,928 lần)
   10. this            (146,350 lần)
   11. that            (135,504 lần)
   12. s               (119,945 lần)
   13. was             (94,077 lần)
   14. movie           (84,278 lần)
   15. as              (84,038 lần)


## 2.6. Mã Hóa Labels

In [ ]:
# Tạo label mapping từ TRAIN
unique_labels = sorted(train_df[LABEL_COLUMN].unique())
label2idx = {label: idx for idx, label in enumerate(unique_labels)}
idx2label = {idx: label for label, idx in label2idx.items()}

print(f'Label mapping:')
for label, idx in label2idx.items():
    train_count = (train_df[LABEL_COLUMN] == label).sum()
    test_count = (test_df[LABEL_COLUMN] == label).sum()
    print(f'   {label} -> {idx} (Train: {train_count:,}, Test: {test_count:,})')

# Mã hóa labels
train_df['label_encoded'] = train_df[LABEL_COLUMN].map(label2idx)
test_df['label_encoded'] = test_df[LABEL_COLUMN].map(label2idx)

print(f'\nĐã mã hóa labels cho train và test')

✅ Label mapping:
   negative -> 0 (Train: 29,997, Test: 5,000)
   positive -> 1 (Train: 29,998, Test: 5,000)

✅ Đã mã hóa labels cho train và test


## 2.7. Mã Hóa Texts

In [ ]:
def encode_text(tokens, word2idx):
    """Mã hóa tokens thành indices"""
    unk_idx = word2idx.get('<UNK>', 1)
    return [word2idx.get(token, unk_idx) for token in tokens]

print('Đang mã hóa texts...\n')

# Encode train
train_df['encoded'] = train_df['tokens'].apply(lambda x: encode_text(x, word2idx))
print(f'Đã mã hóa {len(train_df):,} train samples')
# Encode test (sử dụng CÙNG vocabulary từ train)
test_df['encoded'] = test_df['tokens'].apply(lambda x: encode_text(x, word2idx))
print(f'Đã mã hóa {len(test_df):,} test samples')

# Kiểm tra tỷ lệ UNK trong test
test_unk_count = sum(1 for seq in test_df['encoded'] for idx in seq if idx == 1)
test_total_tokens = sum(len(seq) for seq in test_df['encoded'])
unk_rate = test_unk_count / test_total_tokens * 100

print(f'\nThống kê UNK trong TEST set:')
print(f'   UNK tokens: {test_unk_count:,} / {test_total_tokens:,}')
print(f'   UNK rate: {unk_rate:.2f}%')
print(f'   Tỷ lệ UNK thấp = vocabulary train tốt!')

🔄 Đang mã hóa texts...

✅ Đã mã hóa 59,995 train samples
✅ Đã mã hóa 10,000 test samples

📊 Thống kê UNK trong TEST set:
   UNK tokens: 20,165 / 2,331,919
   UNK rate: 0.86%
   💡 Tỷ lệ UNK thấp = vocabulary train tốt!


## 2.8. Kiểm Tra Encoding/Decoding

In [ ]:
def decode_sequence(sequence, idx2word):
    """Giải mã sequence"""
    words = [idx2word.get(idx, '<UNK>') for idx in sequence]
    return ' '.join(words)

print('Testing encoding/decoding...\n')

# Test với train
print('='*80)
print('TRAIN SAMPLE')
print('='*80)
sample = train_df.sample(1, random_state=42).iloc[0]
print(f"Label: {sample[LABEL_COLUMN]} -> {sample['label_encoded']}")
print(f"\nOriginal (first 150 chars):\n   {sample[TEXT_COLUMN][:150]}...")
print(f"\nEncoded ({len(sample['encoded'])} tokens):\n   {sample['encoded'][:20]}...")
decoded = decode_sequence(sample['encoded'], idx2word)
print(f"\nDecoded:\n   {decoded[:150]}...")
# Test với test
print('\n' + '='*80)
print('TEST SAMPLE')
print('='*80)
sample = test_df.sample(1, random_state=42).iloc[0]
print(f"Label: {sample[LABEL_COLUMN]} -> {sample['label_encoded']}")
print(f"\nOriginal (first 150 chars):\n   {sample[TEXT_COLUMN][:150]}...")
print(f"\nEncoded ({len(sample['encoded'])} tokens):\n   {sample['encoded'][:20]}...")
decoded = decode_sequence(sample['encoded'], idx2word)
print(f"\nDecoded:\n   {decoded[:150]}...")
print('='*80)

🧪 Testing encoding/decoding...

TRAIN SAMPLE
Label: negative -> 0

📝 Original (first 150 chars):
   ***May contain spoilers***<br /><br />I had very high expectations for this film, based on the trailer. I knew a bit about the real Ed Gein, so I figu...

🔢 Encoded (368 tokens):
   [212, 2661, 926, 9, 67, 53, 293, 1202, 18, 11, 19, 413, 23, 2, 1432, 9, 631, 3, 233, 41]...

🔄 Decoded:
   may contain spoilers i had very high expectations for this film based on the trailer i knew a bit about the real ed gein so i figured this was a mediu...

TEST SAMPLE
Label: negative -> 0

📝 Original (first 150 chars):
   This is loosely based on the ideas of the original 80's hit . It's set in the modern day as we see a base in Afghanistan get destroyed by a UAV right ...

🔢 Encoded (955 tokens):
   [11, 7, 3535, 413, 23, 2, 996, 5, 2, 207, 13, 561, 8, 13, 272, 10, 2, 679, 250, 16]...

🔄 Decoded:
   this is loosely based on the ideas of the original s hit it s set in the modern day as we see a base in a

## 2.9. Lưu Dữ Liệu Đã Mã Hóa

In [ ]:
print('Đang lưu dữ liệu đã mã hóa...\n')

# Lưu train encoded
train_texts = train_df['encoded'].values
train_labels = train_df['label_encoded'].values
np.save(os.path.join(OUTPUT_DIR, 'train_encoded_texts.npy'), train_texts, allow_pickle=True)
np.save(os.path.join(OUTPUT_DIR, 'train_encoded_labels.npy'), train_labels)
print(f'Đã lưu train_encoded_texts.npy và train_encoded_labels.npy')

# Lưu test encoded
test_texts = test_df['encoded'].values
test_labels = test_df['label_encoded'].values
np.save(os.path.join(OUTPUT_DIR, 'test_encoded_texts.npy'), test_texts, allow_pickle=True)
np.save(os.path.join(OUTPUT_DIR, 'test_encoded_labels.npy'), test_labels)
print(f'Đã lưu test_encoded_texts.npy và test_encoded_labels.npy')

# Lưu mappings
with open(os.path.join(OUTPUT_DIR, 'word2idx.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=2)
print(f'Đã lưu word2idx.json')

idx2word_str = {str(k): v for k, v in idx2word.items()}
with open(os.path.join(OUTPUT_DIR, 'idx2word.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word_str, f, ensure_ascii=False, indent=2)
print(f'Đã lưu idx2word.json')

with open(os.path.join(OUTPUT_DIR, 'label2idx.json'), 'w', encoding='utf-8') as f:
    json.dump(label2idx, f, ensure_ascii=False, indent=2)
print(f'Đã lưu label2idx.json')

idx2label_str = {str(k): v for k, v in idx2label.items()}
with open(os.path.join(OUTPUT_DIR, 'idx2label.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2label_str, f, ensure_ascii=False, indent=2)
print(f'Đã lưu idx2label.json')

💾 Đang lưu dữ liệu đã mã hóa...

✅ Đã lưu train_encoded_texts.npy và train_encoded_labels.npy
✅ Đã lưu test_encoded_texts.npy và test_encoded_labels.npy
✅ Đã lưu word2idx.json
✅ Đã lưu idx2word.json
✅ Đã lưu label2idx.json
✅ Đã lưu idx2label.json


## 2.10. Lưu Metadata

In [ ]:
# Tạo metadata
train_enc_lengths = train_df['encoded'].apply(len)
test_enc_lengths = test_df['encoded'].apply(len)

metadata = {
    'dataset_info': {
        'train_source': TRAIN_PATH,
        'test_source': TEST_PATH,
        'train_samples': len(train_df),
        'test_samples': len(test_df),
        'num_classes': len(label2idx),
        'train_augmented': True,
        'test_augmented': False
    },
    'vocab_stats': {
        'vocab_size': len(word2idx),
        'built_from': 'train_set_only',
        'min_word_freq': MIN_WORD_FREQ,
        'max_vocab_size': MAX_VOCAB_SIZE,
        'test_unk_rate': float(unk_rate)
    },
    'train_encoding_stats': {
        'min_length': int(train_enc_lengths.min()),
        'max_length': int(train_enc_lengths.max()),
        'mean_length': float(train_enc_lengths.mean()),
        'median_length': float(train_enc_lengths.median()),
        'p95_length': float(train_enc_lengths.quantile(0.95))
    },
    'test_encoding_stats': {
        'min_length': int(test_enc_lengths.min()),
        'max_length': int(test_enc_lengths.max()),
        'mean_length': float(test_enc_lengths.mean()),
        'median_length': float(test_enc_lengths.median()),
        'p95_length': float(test_enc_lengths.quantile(0.95))
    },
    'special_tokens': {
        'pad_token': '<PAD>',
        'pad_idx': 0,
        'unk_token': '<UNK>',
        'unk_idx': 1
    }
}

with open(os.path.join(OUTPUT_DIR, 'metadata.json'), 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f'\nĐã lưu metadata.json')


✅ Đã lưu metadata.json


## 2.11. Tổng Kết

In [ ]:
print('\n' + '='*80)
print('HOÀN THÀNH MÃ HÓA DỮ LIỆU!')
print('='*80)

print(f'\nTổng Kết:')
print(f'   Output directory: {OUTPUT_DIR}/')
print(f'   Vocab size: {len(word2idx):,} (xây dựng từ TRAIN)')
print(f'   Num classes: {len(label2idx)}')

print(f'\nTRAIN SET (ĐÃ AUGMENT):')
print(f'   Samples: {len(train_df):,}')
print(f'   Sequence length: min={train_enc_lengths.min()}, max={train_enc_lengths.max()}, mean={train_enc_lengths.mean():.1f}')
print(f'   Phân phối nhãn:')
for label, count in train_df[LABEL_COLUMN].value_counts().sort_index().items():
    pct = count / len(train_df) * 100
    print(f'      {label}: {count:,} ({pct:.1f}%)')

print(f'\nTEST SET (KHÔNG AUGMENT):')
print(f'   Samples: {len(test_df):,}')
print(f'   Sequence length: min={test_enc_lengths.min()}, max={test_enc_lengths.max()}, mean={test_enc_lengths.mean():.1f}')
print(f'   UNK rate: {unk_rate:.2f}%')
print(f'   Phân phối nhãn:')
for label, count in test_df[LABEL_COLUMN].value_counts().sort_index().items():
    pct = count / len(test_df) * 100
    print(f'      {label}: {count:,} ({pct:.1f}%)')

print(f'\nFiles đã lưu trong {OUTPUT_DIR}/:') 
for file in sorted(os.listdir(OUTPUT_DIR)):
    file_path = os.path.join(OUTPUT_DIR, file)
    size_kb = os.path.getsize(file_path) / 1024
    print(f'   - {file:30s} ({size_kb:>8.1f} KB)')

print(f'\nLƯU Ý QUAN TRỌNG:')
print(f'   Vocabulary chỉ xây dựng từ TRAIN set')
print(f'   Test set sử dụng vocabulary này (có thể có UNK)')
print(f'   Train đã augment, Test KHÔNG augment')
print(f'   Đảm bảo tính khách quan khi đánh giá!')

print('\n' + '='*80)


🎉 HOÀN THÀNH MÃ HÓA DỮ LIỆU!

📊 Tổng Kết:
   ✅ Output directory: ./encoded_split_data/
   ✅ Vocab size: 50,002 (xây dựng từ TRAIN)
   ✅ Num classes: 2

🔵 TRAIN SET (ĐÃ AUGMENT):
   Samples: 59,995
   Sequence length: min=1, max=2485, mean=183.9
   Phân phối nhãn:
      negative: 29,997 (50.0%)
      positive: 29,998 (50.0%)

🔴 TEST SET (KHÔNG AUGMENT):
   Samples: 10,000
   Sequence length: min=6, max=2151, mean=233.2
   UNK rate: 0.86%
   Phân phối nhãn:
      negative: 5,000 (50.0%)
      positive: 5,000 (50.0%)

💾 Files đã lưu trong ./encoded_split_data/:
   - idx2label.json                 (     0.0 KB)
   - idx2word.json                  (  1125.9 KB)
   - label2idx.json                 (     0.0 KB)
   - metadata.json                  (     0.9 KB)
   - test_encoded_labels.npy        (    78.2 KB)
   - test_encoded_texts.npy         (  5460.5 KB)
   - train_encoded_labels.npy       (   468.8 KB)
   - train_encoded_texts.npy        ( 25947.9 KB)
   - word2idx.json                